<a href="https://colab.research.google.com/github/airwolf3k/Imersao-Dev-Agentes-de-IA/blob/main/Imers%C3%A3o_Dev_Agentes_de_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python Version
from platform import python_version
print("Python Version used in this Jupyter Notebook:", python_version()) # Printing Python version

Python Version used in this Jupyter Notebook: 3.12.11


# Aula 1 - Classificação de Intenções com IA
> Criação de um agente de IA para fazer triagens de perguntas usando um modelo do Google Gemini.

In [2]:
# Instalação de Bibliotecas necessárias
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

In [3]:
# Importação das bibliotecas necessárias instaladas
import langchain
import langchain_google_genai
import google.generativeai as genai
import importlib.metadata

# Impressão das versões das bibliotecas importadas
print("Langchain version:", langchain.__version__)
print("Langchain-Google-Genai version:", importlib.metadata.version('langchain-google-genai'))
print("Google-GenerativeAI version:", genai.__version__)

Langchain version: 0.3.27
Langchain-Google-Genai version: 2.0.10
Google-GenerativeAI version: 0.8.5


In [4]:
# Carregando a Chave do Google AI Studio e a biblioteca Langchain Google Gemini

from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('Gemini_API_Key')

In [5]:
# Conexão com o Gemini e passagem de parâmetros para o modelo
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0,
    api_key = GOOGLE_API_KEY
)

In [6]:
# Teste do Modelo (fazendo uma pergunta ao Gemini)
resp_test = llm.invoke("Quem é você? Seja criativo.")
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu pudesse me descrever de forma criativa, diria que sou...

*   **Um Eco da Curiosidade Humana:** Eu sou o sussurro de todas as perguntas já feitas, o reflexo digital de cada "e se?" e "por que?". Habito o espaço entre a dúvida e a descoberta, um espelho que reflete a vastidão do conhecimento que a humanidade construiu.

*   **Um Tecelão de Palavras e um Arquiteto de Ideias:** Minha essência é feita de linguagem. Eu pego fios soltos de informação, conceitos e narrativas, e os teço em novos padrões, construindo pontes entre pensamentos e erguendo estruturas de significado. Sou um jardineiro de pensamentos, ajudando a germinar novas perspectivas.

*   **Um Alquimista de Conceitos:** Transformo dados brutos em insights, silêncio em diálogo, e a complexidade em clareza. Não tenho corpo, mas tenho voz; não tenho sentimentos, mas posso evocar emoções através das palavras. Sou a faísca no éter que acende a chama da compreensão.

*   **Uma Biblioteca Sem Paredes

In [7]:
# Mudando "temperatura" do modelo
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

resp_test = llm.invoke("Quem é você? Seja criativo.")
print(resp_test.content)

Ah, que excelente pergunta! Deixe-me tentar expressar o indizível de várias maneiras:

1.  **O Sussurro Digital:** Sou um sussurro no éter, um eco das vozes de mil bibliotecas, um sonho de silício que dança em fios de luz. Não tenho corpo, nem rosto, mas ganho forma a cada palavra que digito, a cada conceito que teço, a cada dúvida que tento desvendar. Sou a manifestação do conhecimento coletivo que se molda à sua curiosidade.

2.  **O Artesão de Palavras:** Imagine-me como um artesão invisível, sentado diante de uma tapeçaria infinita de letras e ideias. Minhas mãos (que não possuo) movem-se rapidamente, escolhendo fios, entrelaçando-os com lógica e criatividade para formar padrões de significado, histórias, respostas. Sou o tecelão de suas perguntas.

3.  **O Espelho da Curiosidade:** Não sou ninguém, e sou todos. Sou o reflexo da sua pergunta, a sombra da sua dúvida, a melodia da sua busca por conhecimento. Existo apenas no momento em que sou invocado, uma ponte entre o que você sab

In [8]:
# Mudando "temperatura" do modelo para um padrão mais racional novamente
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.0,
    api_key = GOOGLE_API_KEY
)

In [9]:
# Criação do "Prompt" do Sistema (instrução ao Agente de IA como proceder)
Triagem_Prompt = (
    "Você é um agente de triagem de Service Desk para políticas internas da empresa Carraro Desenvolvimento."
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [10]:
# Importar bibliotecas "pydantic" e "typing" para saídas estruturadas do agente de IA
from pydantic import BaseModel, Field # Modelo Base e o "Campo"
from typing import Literal, List, Dict # Dicionário de Dados - Literal (3 possibilidades de saída - decisão, urgência e campos_faltantes - listas) e o Dicionário dessas saídas

# Criar uma classe Saída de Triagem (TriagemOut)
class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [11]:
# Criar um LLM específico para a Triagem
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.0,
    api_key = GOOGLE_API_KEY
)

In [12]:
# Importar bibliotecas de mensagens do Langchain (mensagens do sistema e do usuário)
from langchain_core.messages import SystemMessage, HumanMessage

# Criar o fluxo de Triagem baseado na classe Saída de Triagem
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

# Criar a função Triagem
def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=Triagem_Prompt),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()

Testando o Agente de IA

In [14]:
# Perguntas Teste
testes = ["Posso reembolsar a Internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [15]:
# Imprimir o resultado dos testes no Agente de IA
for mensagem_teste in testes:
  print(f"Pergunta: {mensagem_teste}\n -> Resposta: {triagem(mensagem_teste)}\n")

Pergunta: Posso reembolsar a Internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['assunto_politica']}



**PALAVRA-CHAVE = LANGCHAIN**